# 수치/미들AI/전력 설비 Anomaly detection

code by : Taehoon Kim
- 이 노트북을 차례로 살펴보며 코드의 빈 곳을 채우며 실행하면 수치 과제의 전반적인 과정을 수행해볼 수 있게 제작되었습니다.

## 과제 설명
-  전력설비 이상 탐지 과제

## 데이터 설명
- **입출력**
    - Input: 누적전력량, 유효전력평균, 무효전력평균, 주파수, 전류평균, 상전압평균, 선간전압평균, 온도 등 23개 feature
    - Output:
        - train의 경우 주어지지 않음
        - test의 row별로 '정상' or '비정상'으로 추론
- **데이터 구성**
    - train: 23개 feature, 1,973,449개 row로 이루어진 csv 파일
    - test: 23개 feature, 292,260 row로 이루어진 csv 파일


In [1]:
# 필요한 라이브러리 불러오기

import os
import sys
from datetime import datetime

import torch
from torch import nn
import torch.nn.functional as F
from torch import autograd
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import random as rd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [2]:
USE_CUDA = True

In [3]:
# 시드 고정
seed_value = 1234
rd.seed(seed_value) # set random seed
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    torch.cuda.manual_seed(seed_value) # set pytorch seed GPU

# EDA

In [4]:
### load the dataset into the notebook kernel
train_dataset = pd.read_csv('/workspace/01_data/19_elec/02_processed/d2/train.csv')
print(len(train_dataset))
train_dataset.head()

1973449


,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,...,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,0.0,0.0,0.0,0.00000,101.312500,0.000000,0.000000,24.375,0.0,0.0,...,0.0,0.0,101.5000,0.00,0.00,0.0,0.0,100.68750,0.0,0.00
1,5375707.0,35244.0,19826.0,59.97565,107.385414,125.416664,216.750000,18.750,11988.0,7384.0,...,11020.0,6272.0,101.0625,125.25,216.25,12236.0,6170.0,109.56250,125.0,216.25
2,17781200.0,77056.0,39520.0,59.86300,244.854000,118.083000,205.333000,23.125,25796.0,12244.0,...,26268.0,13572.0,249.1880,118.75,118.75,24992.0,13704.0,242.18800,118.0,118.00
3,10143988.0,0.0,0.0,59.79814,0.000000,133.750000,231.500000,26.875,0.0,0.0,...,0.0,0.0,0.0000,134.50,231.50,0.0,0.0,0.00000,133.0,230.25
4,851769.5,28033.0,14681.0,59.92882,46.687500,225.750000,390.833344,43.125,9642.0,5298.0,...,8897.0,4988.0,45.3125,225.00,389.25,9494.0,4395.0,46.34375,225.5,392.00


In [5]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1973449 entries, 0 to 1973448
Data columns (total 23 columns):
 #   Column  Dtype  
---  ------  -----  
 0   누적전력량   float64
 1   유효전력평균  float64
 2   무효전력평균  float64
 3   주파수     float64
 4   전류평균    float64
 5   상전압평균   float64
 6   선간전압평균  float64
 7   온도      float64
 8   R상유효전력  float64
 9   R상무효전력  float64
 10  R상전류    float64
 11  R상전압    float64
 12  R상선간전압  float64
 13  S상유효전력  float64
 14  S상무효전력  float64
 15  S상전류    float64
 16  S상전압    float64
 17  S상선간전압  float64
 18  T상유효전력  float64
 19  T상무효전력  float64
 20  T상전류    float64
 21  T상전압    float64
 22  T상선간전압  float64
dtypes: float64(23)
memory usage: 346.3 MB


In [6]:
train_dataset.isnull().sum()

누적전력량       16
유효전력평균      16
무효전력평균      16
주파수         16
전류평균        16
상전압평균       16
선간전압평균      16
온도          16
R상유효전력      16
R상무효전력      16
R상전류        16
R상전압        16
R상선간전압      16
S상유효전력    1378
S상무효전력    1378
S상전류      1378
S상전압      1378
S상선간전압    1378
T상유효전력    2896
T상무효전력    2896
T상전류      2896
T상전압      2896
T상선간전압    2896
dtype: int64

In [7]:
test_dataset = pd.read_csv('/workspace/01_data/19_elec/02_processed/d2/test.csv')
print(len(test_dataset))
test_dataset.head()

292260


,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,...,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,1.464900e-02,0.0,0.0,59.89143,0.000000,123.666664,214.583328,18.125,0.0,0.0,...,0.0,0.0,0.0000,124.25,214.75,0.0,0.0,0.0000,123.50,214.75
1,2.766929e+06,0.0,0.0,59.86345,0.000000,264.166656,457.250000,27.500,0.0,0.0,...,0.0,0.0,0.0000,265.50,458.75,0.0,0.0,0.0000,263.75,455.00
2,9.907841e+06,71852.0,43130.0,59.92882,129.041672,215.583328,373.333344,41.250,24834.0,13834.0,...,24290.0,15380.0,132.6250,216.00,372.50,22728.0,13916.0,123.4375,214.75,372.50
3,1.851971e+07,-951.5,-11126.5,59.90100,15.896000,234.583000,405.833000,7.500,-382.5,-3697.0,...,-301.0,-3732.0,15.9690,234.75,234.75,-268.0,-3697.5,15.8750,234.25,234.25
4,5.250714e+06,43470.0,23086.0,59.80745,129.354172,127.000000,219.916672,30.625,14976.0,7992.0,...,14214.0,7724.0,127.4375,127.25,219.75,14280.0,7370.0,126.9375,126.75,219.50


In [8]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292260 entries, 0 to 292259
Data columns (total 23 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   누적전력량   292199 non-null  float64
 1   유효전력평균  292199 non-null  float64
 2   무효전력평균  292199 non-null  float64
 3   주파수     292199 non-null  float64
 4   전류평균    292199 non-null  float64
 5   상전압평균   292199 non-null  float64
 6   선간전압평균  292199 non-null  float64
 7   온도      292199 non-null  float64
 8   R상유효전력  292199 non-null  float64
 9   R상무효전력  292199 non-null  float64
 10  R상전류    292199 non-null  float64
 11  R상전압    292199 non-null  float64
 12  R상선간전압  292199 non-null  float64
 13  S상유효전력  292199 non-null  float64
 14  S상무효전력  292199 non-null  float64
 15  S상전류    292199 non-null  float64
 16  S상전압    292199 non-null  float64
 17  S상선간전압  292199 non-null  float64
 18  T상유효전력  292199 non-null  float64
 19  T상무효전력  292199 non-null  float64
 20  T상전류    292199 non-null  float64
 21  T상전압    29

In [9]:
test_dataset.isnull().sum()

누적전력량     61
유효전력평균    61
무효전력평균    61
주파수       61
전류평균      61
상전압평균     61
선간전압평균    61
온도        61
R상유효전력    61
R상무효전력    61
R상전류      61
R상전압      61
R상선간전압    61
S상유효전력    61
S상무효전력    61
S상전류      61
S상전압      61
S상선간전압    61
T상유효전력    61
T상무효전력    61
T상전류      61
T상전압      61
T상선간전압    61
dtype: int64

# Preprocessing

In [10]:
# 결측치 제거
train_dataset = train_dataset.fillna(method = 'ffill')
train_dataset

,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,...,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,0.00,0.0,0.0,0.000000,101.312500,0.000000,0.000000,24.375,0.0,0.0,...,0.0,0.0,101.50000,0.000,0.000,0.0,0.0,100.68750,0.000,0.000
1,5375707.00,35244.0,19826.0,59.975650,107.385414,125.416664,216.750000,18.750,11988.0,7384.0,...,11020.0,6272.0,101.06250,125.250,216.250,12236.0,6170.0,109.56250,125.000,216.250
2,17781200.00,77056.0,39520.0,59.863000,244.854000,118.083000,205.333000,23.125,25796.0,12244.0,...,26268.0,13572.0,249.18800,118.750,118.750,24992.0,13704.0,242.18800,118.000,118.000
3,10143988.00,0.0,0.0,59.798140,0.000000,133.750000,231.500000,26.875,0.0,0.0,...,0.0,0.0,0.00000,134.500,231.500,0.0,0.0,0.00000,133.000,230.250
4,851769.50,28033.0,14681.0,59.928820,46.687500,225.750000,390.833344,43.125,9642.0,5298.0,...,8897.0,4988.0,45.31250,225.000,389.250,9494.0,4395.0,46.34375,225.500,392.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973444,2995987.25,15761.0,9963.0,59.985000,48.020832,130.333328,225.833328,33.125,5317.0,3585.0,...,5555.0,3361.0,49.68750,131.250,225.750,4889.0,3017.0,45.00000,129.250,224.000
1973445,4097053.25,26416.0,17728.0,59.985000,48.145832,221.250000,383.000000,36.875,10053.0,6581.0,...,8781.0,5669.0,47.21875,221.250,380.750,7582.0,5478.0,42.81250,220.000,383.000
1973446,3244696.25,26644.0,18088.0,59.938175,47.979168,223.750000,387.333344,36.875,10103.0,6776.0,...,9050.0,5886.0,48.40625,224.000,385.500,7491.0,5426.0,41.34375,222.500,387.000
1973447,797771.50,19418.5,21173.0,59.086000,45.208000,213.849000,370.396000,15.938,6532.5,6360.0,...,7110.5,7542.0,48.62500,214.917,214.917,5775.5,7271.0,43.87500,213.603,213.603


In [11]:
test_dataset = test_dataset.fillna(method = 'ffill')
test_dataset

,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,...,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,1.464900e-02,0.0,0.0,59.891430,0.000000,123.666664,214.583328,18.125,0.0,0.0,...,0.0,0.0,0.0000,124.250,214.750,0.0,0.0,0.00000,123.500,214.750
1,2.766929e+06,0.0,0.0,59.863450,0.000000,264.166656,457.250000,27.500,0.0,0.0,...,0.0,0.0,0.0000,265.500,458.750,0.0,0.0,0.00000,263.750,455.000
2,9.907841e+06,71852.0,43130.0,59.928820,129.041672,215.583328,373.333344,41.250,24834.0,13834.0,...,24290.0,15380.0,132.6250,216.000,372.500,22728.0,13916.0,123.43750,214.750,372.500
3,1.851971e+07,-951.5,-11126.5,59.901000,15.896000,234.583000,405.833000,7.500,-382.5,-3697.0,...,-301.0,-3732.0,15.9690,234.750,234.750,-268.0,-3697.5,15.87500,234.250,234.250
4,5.250714e+06,43470.0,23086.0,59.807450,129.354172,127.000000,219.916672,30.625,14976.0,7992.0,...,14214.0,7724.0,127.4375,127.250,219.750,14280.0,7370.0,126.93750,126.750,219.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292255,1.830894e+07,38612.0,55920.0,59.891000,87.667000,257.333000,445.333000,37.500,13276.0,18002.0,...,13182.0,19348.0,90.3750,257.750,257.750,12154.0,18570.0,85.87500,257.250,257.250
292256,9.077721e+06,28910.0,44712.0,59.900765,77.927086,225.333328,390.333344,21.250,8952.0,14328.0,...,9614.0,15666.0,80.5000,225.750,390.500,10344.0,14718.0,79.21875,225.000,389.750
292257,1.371845e+07,42007.0,21831.0,58.517000,75.208000,212.122000,367.406000,10.625,13681.0,6076.0,...,15205.0,7982.0,81.3750,212.287,212.287,13121.0,7773.0,72.37500,212.339,212.339
292258,1.862026e+07,88448.0,27712.0,59.742306,136.750000,226.250000,393.083344,29.375,30564.0,10468.0,...,30560.0,8048.0,138.6250,227.000,392.250,27324.0,9196.0,129.37500,224.750,393.250


In [12]:
# scaling
def scaler(scl, column_name, data1, data2):
    data1[column_name] = scl.fit_transform(data1[column_name].values.reshape(-1,1))
    data2[column_name] = scl.transform(data2[column_name].values.reshape(-1,1))
    
for col in train_dataset.columns:
    scaler(MinMaxScaler(), col, train_dataset, test_dataset)

train_dataset.head()

,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,...,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,0.013780,0.072029,0.059557,0.000000,0.024735,0.000000,0.000000,0.432432,0.409029,0.038310,...,0.297991,0.193380,0.024781,0.000000,0.000000,0.405892,0.116623,0.024582,0.000000,0.000000
1,0.071601,0.209905,0.098486,0.994061,0.026218,0.200480,0.458650,0.351351,0.498273,0.080607,...,0.393077,0.232155,0.024674,0.255352,0.457188,0.474271,0.128003,0.026749,0.139237,0.457914
2,0.205036,0.373476,0.137155,0.992193,0.059780,0.188757,0.434491,0.414414,0.601066,0.108446,...,0.524643,0.277285,0.060838,0.242100,0.251057,0.545556,0.141898,0.059129,0.131440,0.249868
3,0.122889,0.072029,0.059557,0.991118,0.000000,0.213800,0.489861,0.468468,0.409029,0.038310,...,0.297991,0.193380,0.000000,0.274210,0.489429,0.405892,0.116623,0.000000,0.148148,0.487560
4,0.022942,0.181695,0.088383,0.993284,0.011398,0.360863,0.827015,0.702703,0.480808,0.068658,...,0.374758,0.224217,0.011063,0.458716,0.822939,0.458948,0.124729,0.011315,0.251184,0.830069


In [13]:
test_dataset.head()

,누적전력량,유효전력평균,무효전력평균,주파수,전류평균,상전압평균,선간전압평균,온도,R상유효전력,R상무효전력,...,S상유효전력,S상무효전력,S상전류,S상전압,S상선간전압,T상유효전력,T상무효전력,T상전류,T상전압,T상선간전압
0,0.013780,0.072029,0.059557,0.992665,0.000000,0.197682,0.454065,0.342342,0.409029,0.038310,...,0.297991,0.193380,0.000000,0.253313,0.454017,0.405892,0.116623,0.000000,0.137566,0.454738
1,0.043541,0.072029,0.059557,0.992201,0.000000,0.422273,0.967555,0.477477,0.409029,0.038310,...,0.297991,0.193380,0.000000,0.541284,0.969873,0.405892,0.116623,0.000000,0.293790,0.963473
2,0.120349,0.353118,0.144243,0.993284,0.031505,0.344612,0.789985,0.675676,0.593904,0.117553,...,0.507576,0.288463,0.032380,0.440367,0.787526,0.532904,0.142289,0.030137,0.239209,0.788777
3,0.212979,0.068306,0.037710,0.992823,0.003881,0.374983,0.858755,0.189189,0.406181,0.017133,...,0.295394,0.170308,0.003899,0.478593,0.496300,0.404395,0.109803,0.003876,0.260930,0.496030
4,0.070257,0.242086,0.104887,0.991273,0.031581,0.203011,0.465350,0.522523,0.520517,0.084089,...,0.420636,0.241132,0.031113,0.259429,0.464588,0.485694,0.130216,0.030991,0.141186,0.464796


# 모델 설계
### 사용할 파라미터
- `LEARNING_RATE` : 경사하강법(Gradient Descent)을 통해 loss function의 minimum값을 찾아다닐 때, 그 탐색 과정에 있어서의 보폭 정도로 직관적으로 이해 할 수 있습니다. 보폭이 너무 크다면 최적값을 쉽게 지나칠 위험이 있고, 보폭이 너무 작다면 탐색에 걸리는 시간이 길어집니다.
- `EPOCHS` : 
  - 한 번의 epoch는 인공 신경망에서 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것입니다.
  - 즉, epoch이 1만큼 지나면, 전체 데이터 셋에 대해 한번의 학습이 완료된 상태입니다.
  - 모델을 만들 때 적절한 epoch 값을 설정해야만 underfitting과 overfitting을 방지할 수 있습니다.
  - 1 epoch = (데이터 갯수 / batch size) interations

In [14]:
# hyper-parameters
LEARNING_RATE = 1e-3
NUM_EPOCHS = 5
BATCH_SIZE = 128

In [15]:
# implementation of the encoder network
class encoder(nn.Module):
    def __init__(self):

        super(encoder, self).__init__()

        # specify layer 1
        self.encoder_L1 = nn.Linear(in_features=23, out_features=20, bias=True) # add linearity 
        nn.init.xavier_uniform(self.encoder_L1.weight) # init weights according to [9]
        self.encoder_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        self.encoder_L2 = nn.Linear(20, 18, bias=True)
        nn.init.xavier_uniform(self.encoder_L2.weight)
        self.encoder_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L3 = nn.Linear(18, 16, bias=True)
        nn.init.xavier_uniform(self.encoder_L3.weight)
        self.encoder_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L4 = nn.Linear(16, 14, bias=True)
        nn.init.xavier_uniform(self.encoder_L4.weight)
        self.encoder_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L5 = nn.Linear(14, 12, bias=True)
        nn.init.xavier_uniform(self.encoder_L5.weight)
        self.encoder_R5 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L6 = nn.Linear(12, 10, bias=True)
        nn.init.xavier_uniform(self.encoder_L6.weight)
        self.encoder_R6 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L7 = nn.Linear(10, 8, bias=True)
        nn.init.xavier_uniform(self.encoder_L7.weight)
        self.encoder_R7 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L8 = nn.Linear(8, 6, bias=True)
        nn.init.xavier_uniform(self.encoder_L8.weight)
        self.encoder_R8 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.encoder_L9 = nn.Linear(6, 3, bias=True)
        nn.init.xavier_uniform(self.encoder_L9.weight)
        self.encoder_R9 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # init dropout layer with probability p
        self.dropout = nn.Dropout(p=0.0, inplace=True)
        
    def forward(self, x):

        # define forward pass through the network
        x = self.encoder_R1(self.dropout(self.encoder_L1(x)))
        x = self.encoder_R2(self.dropout(self.encoder_L2(x)))
        x = self.encoder_R3(self.dropout(self.encoder_L3(x)))
        x = self.encoder_R4(self.dropout(self.encoder_L4(x)))
        x = self.encoder_R5(self.dropout(self.encoder_L5(x)))
        x = self.encoder_R6(self.dropout(self.encoder_L6(x)))
        x = self.encoder_R7(self.dropout(self.encoder_L7(x)))
        x = self.encoder_R8(self.dropout(self.encoder_L8(x)))
        x = self.encoder_R9(self.encoder_L9(x)) # don't apply dropout to the AE bottleneck

        return x

In [16]:
# implementation of the decoder network
class decoder(nn.Module):

    def __init__(self):

        super(decoder, self).__init__()

        # specify layer
        self.decoder_L1 = nn.Linear(in_features=3, out_features=6, bias=True) # add linearity 
        nn.init.xavier_uniform(self.decoder_L1.weight)  # init weights according to [9]
        self.decoder_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        self.decoder_L2 = nn.Linear(6, 8, bias=True)
        nn.init.xavier_uniform(self.decoder_L2.weight)
        self.decoder_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.decoder_L3 = nn.Linear(8, 10, bias=True)
        nn.init.xavier_uniform(self.decoder_L3.weight)
        self.decoder_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.decoder_L4 = nn.Linear(10, 12, bias=True)
        nn.init.xavier_uniform(self.decoder_L4.weight)
        self.decoder_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.decoder_L5 = nn.Linear(12, 14, bias=True)
        nn.init.xavier_uniform(self.decoder_L5.weight)
        self.decoder_R5 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.decoder_L6 = nn.Linear(14, 16, bias=True)
        nn.init.xavier_uniform(self.decoder_L6.weight)
        self.decoder_R6 = nn.LeakyReLU(negative_slope=0.4, inplace=True)
        
        self.decoder_L7 = nn.Linear(16, 18, bias=True)
        nn.init.xavier_uniform(self.decoder_L7.weight)
        self.decoder_R7 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.decoder_L8 = nn.Linear(18, 20, bias=True)
        nn.init.xavier_uniform(self.decoder_L8.weight)
        self.decoder_R8 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.decoder_L9 = nn.Linear(20, 23, bias=True)
        nn.init.xavier_uniform(self.decoder_L9.weight)
        self.decoder_R9 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.dropout = nn.Dropout(p=0.0, inplace=True)

    def forward(self, x):

        # define forward pass through the network
        x = self.decoder_R1(self.dropout(self.decoder_L1(x)))
        x = self.decoder_R2(self.dropout(self.decoder_L2(x)))
        x = self.decoder_R3(self.dropout(self.decoder_L3(x)))
        x = self.decoder_R4(self.dropout(self.decoder_L4(x)))
        x = self.decoder_R5(self.dropout(self.decoder_L5(x)))
        x = self.decoder_R6(self.dropout(self.decoder_L6(x)))
        x = self.decoder_R7(self.dropout(self.decoder_L7(x)))
        x = self.decoder_R8(self.dropout(self.decoder_L8(x)))
        x = self.decoder_R9(self.decoder_L9(x)) # don't apply dropout to the AE output
        
        return x

# 학습

In [17]:
# init training network classes / architectures
encoder_train = encoder()
decoder_train = decoder()

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    encoder_train = encoder().cuda()
    decoder_train = decoder().cuda()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/pyt

In [18]:
# define the optimization criterion / loss function
loss_function = F.mse_loss

# define learning rate and optimization strategy
encoder_optimizer = torch.optim.Adam(encoder_train.parameters(), lr=LEARNING_RATE)
decoder_optimizer = torch.optim.Adam(decoder_train.parameters(), lr=LEARNING_RATE)

In [19]:
# convert pre-processed data to pytorch tensor
train_dataset = torch.from_numpy(train_dataset.values).float()

# convert to pytorch tensor - none cuda enabled
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
# note: we set num_workers to zero to retrieve deterministic results

# determine if CUDA is available at compute node
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    dataloader = DataLoader(train_dataset.cuda(), batch_size=BATCH_SIZE, shuffle=True)

In [20]:
# init collection of mini-batch losses
losses = []

# convert encoded transactional data to torch Variable
data = autograd.Variable(train_dataset)

# train autoencoder model
for epoch in range(NUM_EPOCHS):

    # init mini batch counter
    mini_batch_count = 0
    
    # determine if CUDA is available at compute node
    if(torch.backends.cudnn.version() != None) and (USE_CUDA == True):
        
        # set networks / models in GPU mode
        encoder_train.cuda()
        decoder_train.cuda()

    # set networks in training mode (apply dropout when needed)
    encoder_train.train()
    decoder_train.train()

    # start timer
    start_time = datetime.now()
        
    # iterate over all mini-batches
    for mini_batch_data in dataloader:

        # increase mini batch counter
        mini_batch_count += 1

        # convert mini batch to torch variable
        mini_batch_torch = autograd.Variable(mini_batch_data)

        # run forward pass
        z_representation = encoder_train(mini_batch_torch) # encode mini-batch data
        mini_batch_reconstruction = decoder_train(z_representation) # decode mini-batch data
        
        # determine reconstruction loss
        reconstruction_loss = loss_function(mini_batch_reconstruction, mini_batch_torch)
        
        # reset graph gradients
        decoder_optimizer.zero_grad()
        encoder_optimizer.zero_grad()

        # run backward pass
        reconstruction_loss.backward()
        
        # update network parameters
        decoder_optimizer.step()
        encoder_optimizer.step()

        # print training progress each 1'000 mini-batches
        if mini_batch_count % 1000 == 0:
            
            # print the training mode: either on GPU or CPU
            mode = 'GPU' if (torch.backends.cudnn.version() != None) and (USE_CUDA == True) else 'CPU'
            
            # print mini batch reconstuction results
            now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            end_time = datetime.now() - start_time
            print('[LOG {}] training status, epoch: [{:04}/{:04}], batch: {:04}, loss: {}, mode: {}, time required: {}'.format(now, (epoch+1), NUM_EPOCHS, mini_batch_count, np.round(reconstruction_loss.cpu().data, 4), mode, end_time))

            # reset timer
            start_time = datetime.now()

    # =================== evaluate model performance =============================
    
    # set networks in evaluation mode (don't apply dropout)
    encoder_train.cpu().eval()
    decoder_train.cpu().eval()

    # reconstruct encoded transactional data
    reconstruction = decoder_train(encoder_train(data))
    
    # determine reconstruction loss - all transactions
    reconstruction_loss_all = loss_function(reconstruction, data)
            
    # collect reconstruction loss
    losses.extend([reconstruction_loss_all.data])
    
    # print reconstuction loss results
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] training status, epoch: [{:04}/{:04}], loss: {:.10f}'.format(now, (epoch+1), NUM_EPOCHS, reconstruction_loss_all.data))

    # =================== save model snapshot to disk ============================
    
    # save trained encoder model file to disk
    now = datetime.utcnow().strftime("%Y%m%d-%H_%M_%S")
    encoder_model_name = "{}_ep_{}_encoder_model.pth".format(now, (epoch+1))
    torch.save(encoder_train.state_dict(), os.path.join("/workspace/02_workspace/19_elec/models", encoder_model_name))

    # save trained decoder model file to disk
    decoder_model_name = "{}_ep_{}_decoder_model.pth".format(now, (epoch+1))
    torch.save(encoder_train.state_dict(), os.path.join("/workspace/02_workspace/19_elec/models", decoder_model_name))

[LOG 20211116-06:29:23] training status, epoch: [0001/0005], batch: 1000, loss: 0.002899999963119626, mode: GPU, time required: 0:00:12.381231
[LOG 20211116-06:29:35] training status, epoch: [0001/0005], batch: 2000, loss: 0.002400000113993883, mode: GPU, time required: 0:00:12.213774
[LOG 20211116-06:29:47] training status, epoch: [0001/0005], batch: 3000, loss: 0.0026000000070780516, mode: GPU, time required: 0:00:12.190551
[LOG 20211116-06:29:59] training status, epoch: [0001/0005], batch: 4000, loss: 0.0020000000949949026, mode: GPU, time required: 0:00:12.011094
[LOG 20211116-06:30:11] training status, epoch: [0001/0005], batch: 5000, loss: 0.0013000000035390258, mode: GPU, time required: 0:00:12.107470
[LOG 20211116-06:30:24] training status, epoch: [0001/0005], batch: 6000, loss: 0.0010999999940395355, mode: GPU, time required: 0:00:12.158208
[LOG 20211116-06:30:36] training status, epoch: [0001/0005], batch: 7000, loss: 0.0008999999845400453, mode: GPU, time required: 0:00:12.1

# 추론
테스트 데이터의 타겟 변수를 submit 양식에 맞춰 저장한 파일을 aiconnect 플랫폼을 통해 제출하면 추론 점수를 확인할 수 있습니다.

answer 컬럼 값을 여러분의 모델의 추론 결과로 채워 제출 파일을 만듭니다 (현재는 모두 동일한 값으로 채워져 있습니다).

ID값을 기준으로 채점을 진행하는 점 유의해주시기 바랍니다.

In [21]:
# convert pre-processed data to pytorch tensor
test_dataset = torch.from_numpy(test_dataset.values).float()

# convert to pytorch tensor - none cuda enabled
dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
# note: we set num_workers to zero to retrieve deterministic results

# determine if CUDA is available at compute node
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    dataloader = DataLoader(test_dataset.cuda(), batch_size=BATCH_SIZE, shuffle=True)

In [22]:
# restore pretrained model checkpoint
encoder_model_name = "20211112-00_04_47_ep_3_encoder_model.pth"
decoder_model_name = "20211112-00_04_47_ep_3_decoder_model.pth"

# init training network classes / architectures
encoder_eval = encoder()
decoder_eval = decoder()

# load trained models
encoder_eval.load_state_dict(torch.load(os.path.join("/workspace/02_workspace/19_elec", encoder_model_name)))
decoder_eval.load_state_dict(torch.load(os.path.join("/workspace/02_workspace/19_elec", decoder_model_name)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/opt/conda/lib/pyt

<All keys matched successfully>

In [23]:
# convert encoded transactional data to torch Variable
data = autograd.Variable(test_dataset)

In [24]:
# reconstruct encoded data
reconstruction = decoder_eval(encoder_eval(data))

In [25]:
data

tensor([[0.0138, 0.0720, 0.0596,  ..., 0.0000, 0.1376, 0.4547],
        [0.0435, 0.0720, 0.0596,  ..., 0.0000, 0.2938, 0.9635],
        [0.1203, 0.3531, 0.1442,  ..., 0.0301, 0.2392, 0.7888],
        ...,
        [0.1613, 0.2364, 0.1024,  ..., 0.0177, 0.2365, 0.4496],
        [0.2141, 0.4180, 0.1140,  ..., 0.0316, 0.2503, 0.8327],
        [0.0666, 0.2112, 0.0658,  ..., 0.0131, 0.2501, 0.8280]])

In [26]:
len(data)

292260

In [27]:
data[0]

tensor([0.0138, 0.0720, 0.0596, 0.9927, 0.0000, 0.1977, 0.4541, 0.3423, 0.4090,
        0.0383, 0.0000, 0.2523, 0.4563, 0.2980, 0.1934, 0.0000, 0.2533, 0.4540,
        0.4059, 0.1166, 0.0000, 0.1376, 0.4547])

In [28]:
len(data[0])

23

In [29]:
reconstruction[292137]

tensor([ 0.0582,  0.0769,  0.0605,  0.9943, -0.0013,  0.3750,  0.8541,  0.5190,
         0.4086,  0.0352, -0.0013,  0.4782,  0.8536,  0.3023,  0.1921, -0.0026,
         0.4739,  0.8480,  0.4072,  0.1190, -0.0016,  0.2600,  0.8456],
       grad_fn=<SelectBackward>)

In [30]:
loss_function(reconstruction[0], data[0])

tensor(5.1131e-05, grad_fn=<MseLossBackward>)

In [31]:
loss_list = []

for i in range(len(data)):
    loss = loss_function(reconstruction[i], data[i])
    loss_list.append(loss)

In [32]:
loss_list[:5]

[tensor(5.1131e-05, grad_fn=<MseLossBackward>),
 tensor(1.5964e-05, grad_fn=<MseLossBackward>),
 tensor(4.3192e-05, grad_fn=<MseLossBackward>),
 tensor(0.0022, grad_fn=<MseLossBackward>),
 tensor(6.0644e-05, grad_fn=<MseLossBackward>)]

In [33]:
reversed_list = sorted(loss_list, reverse=True)
reversed_list[:5]

[tensor(0.0206, grad_fn=<MseLossBackward>),
 tensor(0.0178, grad_fn=<MseLossBackward>),
 tensor(0.0177, grad_fn=<MseLossBackward>),
 tensor(0.0175, grad_fn=<MseLossBackward>),
 tensor(0.0169, grad_fn=<MseLossBackward>)]

In [34]:
reversed_list[int(len(reversed_list)*0.1)]

tensor(0.0010, grad_fn=<MseLossBackward>)

In [35]:
submit = pd.read_csv('/workspace/02_workspace/19_elec/sample_submission.csv')
submit.head()

,ID,label
0,0,정상
1,1,정상
2,2,정상
3,3,정상
4,4,정상


In [36]:
loss_list_ = []
for i in range(len(loss_list)):
    if float(loss_list[i]) > 0.0009:
        loss_list_.append('비정상')
    else:
        loss_list_.append('정상')

In [37]:
loss_list_[:5]

['정상', '정상', '정상', '비정상', '정상']

In [38]:
submit['label'] = loss_list_

In [39]:
submit.head()

,ID,label
0,0,정상
1,1,정상
2,2,정상
3,3,비정상
4,4,정상


In [40]:
submit['label'].value_counts()

정상     259144
비정상     33116
Name: label, dtype: int64

In [41]:
submit.to_csv('/workspace/02_workspace/19_elec/prediction.csv', index=False)